In [23]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S2'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #CHange from 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 100
Total Tokens Found = 1247
Total Tokens Output = 1247
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 100
Total Tokens= 1247
Unique Tokens= 464
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 100
Sorted Token Size = 464
Clean Token Size = 166
*Stop Replacements here
References Processed =  100
Total Replacement Pairs = 3
Tokens Read = 1247
Tokens Changed =  8
References Changed = 8


In [24]:
status="done"

In [25]:
print(status)

done


In [26]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [27]:
import numpy as np

Create a function of the parameters that returns the F score

In [28]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [29]:
entropy_F1(mu=0.5,muIncr=0.1,beta=6,sigma=7,epsilon=4.7,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 4.7
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 205
NewMatrix Iterateblocks Total Time = 0.2938821520656347


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 4.7
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 75
NewMatrix Iterateblocks Total Time = 0.1988878734409809


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 4.7
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 33
NewMatrix Iterateblocks Total Time = 0.15063156094402075


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 4.7
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 14
NewMatrix Iterateblocks Total Time = 0.13816524855792522

Precision = TP / (TP + FP) ......................... = 0.87500000
Recall = TP / (TP + FN) ............................ = 0.87500000
F-

0.875

Create functions for variable transformation.

In [30]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [31]:
inv_softplus(1)

0.541324854612918

In [32]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [33]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [34]:
import os, sys

In [35]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [36]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [37]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [38]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.7586617742112225

In [39]:
optimizer=None

In [40]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [41]:
from bayes_opt import SequentialDomainReductionTransformer

In [42]:
bounds_transformer = SequentialDomainReductionTransformer()

In [43]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [44]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(6), 
#             "sigma_sp": inv_softplus(7),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(4.7)},
#     lazy=True,
# )

In [45]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  1.743    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  1.792    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        |  0.4418   |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        |  1.689    |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        |  1.447    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  1.959    |  62.55    |  149.6    | -3.623    |  2.072    |  407.7    |
Mu:0.88|Mu+:0.03|Beta:73.00|Sigma:407.00|Epsilon:149.17|EpIter:0.00
|  40       |  1.959    |  73.37    |  149.2    | -3.588    |  2.008    |  407.3    |
Mu:0.89|Mu+:0.03|Beta:63.00|Sigma:400.00|Epsilon:148.81|EpIter:0.00
|  41       |  1.959    |  63.63    |  148.8    | -3.62     |  2.066    |  400.2    |
Mu:0.89|Mu+:0.03|Beta:72.00|Sigma:400.00|Epsilon:142.65|EpIter:0.00
|  42       |  1.959    |  72.4     |  142.6    | -3.618    |  2.063    |  400.6    |
Mu:0.89|Mu+:0.03|Beta:64.00|Sigma:406.00|Epsilon:142.94|EpIter:0.00
|  43       |  1.959    |  64.5     |  142.9    | -3.617    |  2.06     |  406.3    |
Mu:0.89|Mu+:0.03|Beta:71.00|Sigma:405.00|Epsilon:143.20|EpIter:0.00
|  44       |  1.959    |  71.61    |  143.2    | -3.615    |  2.058    |  406.0    |
Mu:0.89|Mu+:0.03|Beta:65.00|Sigma:401.00|Epsilon:147.69|EpIter:0.00
|  45       |  1.959    |  65.21    |  147.7    | -3.614    |  2.056    |  4

In [46]:
optimizer.max

{'target': 1.9588135134366762,
 'params': {'beta_sp': 68.2423484528834,
  'epsilon_sp': 145.56699708347966,
  'muIncr_logit': -3.604465057328993,
  'mu_logit': 2.0381497294623347,
  'sigma_sp': 403.55440834120367}}

In [47]:
inv_logit(optimizer.max["target"])

0.87640449

In [48]:
params=optimizer.max["params"]

In [49]:
softplus(params["beta_sp"])

68.2423484528834

In [50]:
softplus(params["sigma_sp"])

403.55440834120367

In [51]:
inv_logit(params["mu_logit"])

0.8847447273744214

In [52]:
inv_logit(params["muIncr_logit"])

0.026481639082321012

In [53]:
softplus(params["epsilon_sp"])

145.56699708347966

In [54]:
entropy_F1(mu=0.885,
           muIncr=0.0265,
           beta=68,
           sigma=403,
           epsilon=145.57,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.885
Starting Iteration epsilon= 145.57
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 228
NewMatrix Iterateblocks Total Time = 0.6792284930124879

Precision = TP / (TP + FP) ......................... = 0.95121951
Recall = TP / (TP + FN) ............................ = 0.81250000
F-Measure .......................................... = 0.87640449


0.87640449